# Intrasubject Compression
Author: Spencer Weston

This notebook intends to analyze how we can/should compress the outputs from the intrasubject join made in `s3_combine_intrasubject_data.py`. Remember the form of the join. We have a one to many join from entry_id's to treatment and device status id's. Thus, we have duplicate entry ID's and blood glucose entries. We will have one or more treatment and/or device status row associated with each entry id. 

We need to perform to compress data column-wise and row-wise:
* Column-wise compression - select relevant columns and compress them through an aggregation (sum, mean, first, last, etc.) as appropriate. Initially, this will just involve carbs and insulin
* Row-wise compression - Effectively, this will be a group-by and aggregate operation. We will groupby on entry_id to remove duplicate entry_id's then select the best aggregation for each variable. For example, we can take the first blood glucose entry since they're identical but we may want to take a sum of insulin.

In [2]:
import os
import numpy as np
import pandas as pd 
import re 
from datetime import datetime, timedelta, timezone
import pickle
import matplotlib.pyplot as plt
import psycopg2

import pytz
from collections import namedtuple
from bgpredict.s3_combine_intra_subject_data import S3Connection
# from bgpredict.intrasubject_compression import SubjectCompressor 

First, we're going to get some aggregate statistics for each column across all the tables. We performed these aggregations on an EC2 instance and wrote the results to a pickle file in the `misc` folder. The code had a bug where some tables were processed twice, so we have to do a little work to clean those up.

In [11]:
with open('../../misc/column_stats.pickle', 'rb') as file:
    col_stats = pickle.load(file)
    
print(len(col_stats))

4388


In [12]:
# We can use the subject id to identify and drop the duplicates
entries = col_stats['entryid']
new_list = []
updated_col_stats= {}
for k, v in col_stats.items():
    new_list = []
    for i in v:
        s_id = i['subjectid']
        if i not in new_list:
            new_list.append(i)
    updated_col_stats[k] = new_list

In [13]:
# write out the update result
# with open('../../misc/column_stats.pickle', 'wb') as file:
#     pickle.dump(updated_col_stats, file)
# with open('../../misc/column_stats.pickle', 'rb') as file:
#     col_stats = pickle.load(file)

In [14]:
columns = list(col_stats.keys())
# columns
col_by_freq = [(c, col_stats[c], len(col_stats[c])) for c in columns]
col_by_freq.sort(reverse= True, key=lambda x: x[2])
for x in col_by_freq[:25]:
    print(x[0], x[2])

entryid 141
devicestatusid 141
treatmentid 141
enteredBy 141
insulin 141
carbs 141
eventType 141
time 140
duration 140
subjectid 140
timestamp_y 137
device 137
created_at_z 137
_id_z 137
timestamp_ds 137
_id_tre 137
created_at_tre 137
timestamp 137
pump/clock 136
notes 136
pump/reservoir 135
rate 134
absolute 134
reason 133
glucoseType 133


If we look at the top 25 columns, we see that every table has a "insulin" and "carbs" column. Thus, we can pretty simply just take the insulin and carb columns. However, it's weird that only 137/141 tables have the `timestamp_y` column which *should* have been created in the join. 

In [15]:
for x in col_by_freq:
    if 'bg' in x[0]:
        if x[2] > 1:
            print(x[0], x[2])

openaps/suggested/bg 124
openaps/enacted/bg 116
wizard/bg_target_low 98
wizard/bg 98
wizard/bg_target_high 98
bg_tre 78
bg_z 77
stale/bg_targets/0/low 77
stale/bg_targets/0/offset 77
stale/bg_targets/0/_offset 77
stale/bg_targets/0/high 77
stale/bg_targets/1/low 77
stale/bg_targets/1/offset 77
stale/bg_targets/1/_offset 77
stale/bg_targets/1/high 77
stale/bg_targets/2/low 77
stale/bg_targets/2/offset 77
stale/bg_targets/2/_offset 77
stale/bg_targets/2/high 77
stale/bg_targets/3/low 77
stale/bg_targets/3/offset 77
stale/bg_targets/3/_offset 77
stale/bg_targets/3/high 77
stale/bg_targets/4/low 77
stale/bg_targets/4/offset 77
stale/bg_targets/4/_offset 77
stale/bg_targets/4/high 77
stale/bg_targets/5/low 77
stale/bg_targets/5/offset 77
stale/bg_targets/5/_offset 77
stale/bg_targets/5/high 77
stale/bg_targets/6/low 77
stale/bg_targets/6/offset 77
stale/bg_targets/6/_offset 77
stale/bg_targets/6/high 77
stale/bg_targets/7/low 77
stale/bg_targets/7/offset 77
stale/bg_targets/7/_offset 77
sta

## Single Table Compression
Let's do compression on a random table

In [123]:
os.getcwd()

'C:\\Users\\spenc\\Documents\\Berkeley\\Capstone\\BGPredict\\bgpredict\\Notebooks'

In [124]:
path = "C:\\Users\\spenc\\Documents\\Berkeley\\Capstone\\intrasubject_joinv0.2\\309157_joined_csv"
print(os.path.exists(path))
df = pd.read_csv(path)
df.shape

True


C:\Users\spenc\AppData\Local\Temp\ipykernel_14028\486633859.py:3: DtypeWarning: Columns (4,103,191,232,235,237,397,400,402,404,701,703,704,706,708,710,711,712,713,717,719,728,733,734,735,736,739,741,742,743,745,748,752,758,759,760,761,762) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


(162931, 766)

In [125]:
cols = ['entryid', 'subjectid', 'timestamp_y', 'bg', 'insulin', 'carbs']
compressed_df = df.loc[:, cols]
compressed_df.head(10)

,entryid,subjectid,timestamp_y,bg,insulin,carbs
0,121949,309157,2019-02-01 13:21:18.909000+10:00,69,NaN,NaN
1,121949,309157,2019-02-01 13:21:18.909000+10:00,69,NaN,NaN
2,121949,309157,2019-02-01 13:21:18.909000+10:00,69,NaN,NaN
3,121764,309157,2019-02-01 13:36:12+00:00,199,NaN,NaN
4,121764,309157,2019-02-01 13:36:12+00:00,199,NaN,NaN
5,121764,309157,2019-02-01 13:36:12+00:00,199,NaN,NaN
6,121762,309157,2019-02-01 13:41:12+00:00,204,NaN,NaN
7,121762,309157,2019-02-01 13:41:12+00:00,204,NaN,NaN
8,121762,309157,2019-02-01 13:41:12+00:00,204,NaN,NaN
9,121762,309157,2019-02-01 13:41:12+00:00,204,NaN,NaN


In [126]:
compressed_df.loc[~compressed_df['carbs'].isna(), :].head(10)

,entryid,subjectid,timestamp_y,bg,insulin,carbs
20938,108749,309157,2019-02-26 03:17:19.544000+10:00,117,0.00,0.0
20939,108749,309157,2019-02-26 03:17:19.544000+10:00,117,0.00,0.0
20945,108745,309157,2019-02-26 03:27:19.069000+10:00,120,1.70,10.0
20946,108745,309157,2019-02-26 03:27:19.069000+10:00,120,1.70,10.0
21007,108462,309157,2019-02-26 05:03:27+00:00,63,1.73,15.0
21008,108462,309157,2019-02-26 05:03:27+00:00,63,1.73,15.0
21104,108403,309157,2019-02-26 07:33:27+00:00,61,3.00,30.0
21105,108403,309157,2019-02-26 07:33:27+00:00,61,3.00,30.0
21106,108403,309157,2019-02-26 07:33:27+00:00,61,3.00,30.0
21125,108389,309157,2019-02-26 08:03:26+00:00,94,0.00,0.0


In [130]:
def select_timestamp(x):
    x = x.loc[~x.isna()]
    if len(x) ==0:
        raise Exception
    else:
        return x.iloc[0]
ins_count = {'sum': 0, 'first': 0}
carb_count = {'sum': 0, 'first': 0}
def select_insulin(x):
    x = x.loc[~x.isna()]
    if len(x) > 1:
        # For subject 309157, there are often duplicate entry values. These are *obviously* duplicates
        # as validated by looking at the original treatment table. However, this may not always 
        # be the case. A possible improved validation method would check the treatment 'created_at'
        # or timestamp value and verify the insulin was added at the same time
        if len(pd.unique(x)) == 1:
            ins_count['first'] +=1
            return x.iloc[0]
        else:
            ins_count['sum'] += 1
            return x.sum()
    elif x.empty:
        return 0
    else:
        return x
    
def select_carbs(x):
    x = x.loc[~x.isna()]
    if len(x) > 1:
        # For subject 309157, there are often duplicate entry values. These are *obviously* duplicates
        # as validated by looking at the original treatment table. However, this may not always 
        # be the case. A possible improved validation method would check the treatment 'created_at'
        # or timestamp value and verify the insulin was added at the same time
        if len(pd.unique(x)) == 1:
            carb_count['first'] +=1
            return x.iloc[0]
        else:
            carb_count['sum'] += 1
            return x.sum()
    elif x.empty:
        return 0
    else:
        return x

test =compressed_df.groupby('entryid', dropna=False).agg({'subjectid': "first",
                                                         'timestamp_y': select_timestamp,
                                                         'bg': 'first',
                                                         'insulin': select_insulin,
                                                         'carbs': select_carbs})
print(f"insulin count: {ins_count}, carb-count: {carb_count}") 

insulin count: {'sum': 83, 'first': 1713}, carb-count: {'sum': 75, 'first': 1442}


In [131]:
test =test.sort_values('timestamp_y')
test.columns = ['subjectid', 'timestamp', 'bg', 'insulin', 'carbs']
test.reindex()

,subjectid,timestamp,bg,insulin,carbs
entryid,,,,,
121949,309157,2019-02-01 13:21:18.909000+10:00,69,0.0,0.0
121764,309157,2019-02-01 13:36:12+00:00,199,0.0,0.0
121762,309157,2019-02-01 13:41:12+00:00,204,0.0,0.0
121760,309157,2019-02-01 13:46:12+00:00,208,0.0,0.0
121758,309157,2019-02-01 13:51:12+00:00,215,0.0,0.0
...,...,...,...,...,...
7,309157,2019-08-30 23:40:41+00:00,126,0.0,0.0
5,309157,2019-08-30 23:45:41+00:00,127,0.0,0.0
3,309157,2019-08-30 23:50:41+00:00,128,0.0,0.0


## Generalized Compression Considerations

The previous section showed that the compression, for this specific table, is not particularly hard. However, there are several considerations we need to account for when generalizing the compression.
* Only 64 tables have a `bg` column. 77 tables have a `bg_z` table. 64+77 = 141 = the number of tables. If each table has a `bg` OR `bg_z`, we can just select whatever blood glucose column exists. Otherwise, we'll need to figure out if that data was lost in the join.
* 137/141 tables have a `timestamp` and/or `timestamp_y` column. If all tables have a `timestamp` OR `timestamp_y` column, we can select the column that exists. Otherwise, we'll need to check rather other timestamps exist.

In [16]:
col_by_subj = pd.read_csv('../../misc/column_by_subject.csv')
col_by_subj.rename({"Unnamed: 0" : 'subid'}, axis=1, inplace=True)
col_by_subj.head()

,subid,entryid,devicestatusid,treatmentid,time,bg,timestamp_y,device,created_at_z,_id_z,...,appended/1/data/31/rate,appended/1/data/32/offset,appended/1/data/32/rate,appended/1/data/33/offset,appended/1/data/33/rate,appended/1/data/34/offset,appended/1/data/34/rate,appended/1/data/35/offset,appended/1/data/35/rate,ALEXA
0,12689381,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,13029224,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,13484299,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1352464,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,13783771,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Check if every table has a bg column
test = col_by_subj.loc[(col_by_subj['bg']==1)|(col_by_subj['bg_z'] ==1) ,:]
s = len(test)
print(f"{s} tables have bg column")
col_by_subj.loc[(col_by_subj['bg']==0)&(col_by_subj['bg_z'] ==0) ,:]

140 tables have bg column


,subid,entryid,devicestatusid,treatmentid,time,bg,timestamp_y,device,created_at_z,_id_z,...,appended/1/data/31/rate,appended/1/data/32/offset,appended/1/data/32/rate,appended/1/data/33/offset,appended/1/data/33/rate,appended/1/data/34/offset,appended/1/data/34/rate,appended/1/data/35/offset,appended/1/data/35/rate,ALEXA
5,14092221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Every table has a blood glucose column. The outlier is 14092221 which wasn't computed due to its large size.

In [22]:
# Check if every table has a timestamp column
test = col_by_subj.loc[(col_by_subj['timestamp']==1)|(col_by_subj['timestamp_y'] ==1) ,:]
s = len(test)
print(f"{s} tables have timestamp column")
col_by_subj.loc[(col_by_subj['timestamp']==0)&(col_by_subj['timestamp_y'] ==0) ,:]

136 tables have timestamp column


,subid,entryid,devicestatusid,treatmentid,time,bg,timestamp_y,device,created_at_z,_id_z,...,appended/1/data/31/rate,appended/1/data/32/offset,appended/1/data/32/rate,appended/1/data/33/offset,appended/1/data/33/rate,appended/1/data/34/offset,appended/1/data/34/rate,appended/1/data/35/offset,appended/1/data/35/rate,ALEXA
5,14092221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,42052178,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,50311906,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,61179686,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,66773091,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
test = test.loc[test.subid == 22961398, :]
cols = [c for c in list(test.columns) if 'timestamp' in c]

In [24]:
cols

['timestamp_y',
 'loop/timestamp',
 'loop/iob/timestamp',
 'loop/enacted/timestamp',
 'loop/cob/timestamp',
 'uploader/timestamp',
 'loop/recommendedTempBasal/timestamp',
 'timestamp_ds',
 'timestamp',
 'preferences/timestamp',
 'openaps/suggested/timestamp',
 'openaps/enacted/timestamp',
 'openaps/iob/timestamp',
 'openaps/iob/lastTemp/timestamp',
 'pump/status/timestamp',
 'bolus/timestamp',
 'wizard/timestamp',
 'raw_rate/timestamp',
 'raw_duration/timestamp',
 'pump/iob/timestamp',
 'square/timestamp',
 'xdripjs/timestamp',
 'override/timestamp',
 'mmtune/timestamp',
 'timestamp_z',
 'timestamp_tre']

Here, we have 4 tables (not 1409221) without a timestamp. Let's see if we can find a timestamp column.

In [32]:
(col_by_subj[col_by_subj.columns] != 0).any()

subid                        True
entryid                      True
devicestatusid               True
treatmentid                  True
time                         True
                             ... 
appended/1/data/34/offset    True
appended/1/data/34/rate      True
appended/1/data/35/offset    True
appended/1/data/35/rate      True
ALEXA                        True
Length: 4388, dtype: bool

In [34]:
subs = col_by_subj.loc[(col_by_subj['timestamp']==0)&(col_by_subj['timestamp_y'] ==0) , 'subid']
test = col_by_subj.loc[(col_by_subj.subid.isin(subs)), ]
test = test.loc[:, (test[test.columns] != 0).any()]
test.head()

,subid,entryid,devicestatusid,treatmentid,time,bg,temp,duration,enteredBy,rate,...,created_at,_id,isECarbs,boluscalc/iobused,boluscalc/percentage,target,timestamp_tre,tisf,Meg's Phone,Megs Phone
5,14092221,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,42052178,1,1,1,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,0,0
60,50311906,1,1,1,1,0,0,1,1,1,...,1,1,0,0,0,0,1,0,0,0
68,61179686,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,1,0,1,1
77,66773091,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,1,0,0,0


In [36]:
cols = [c for c in list(test.columns) if 'timestamp' in c]
test[cols]

,bolus/timestamp,wizard/timestamp,raw_rate/timestamp,raw_duration/timestamp,square/timestamp,timestamp_z,timestamp_tre
5,0,0,0,0,0,0,0
49,1,1,1,1,1,1,1
60,1,1,1,1,0,1,1
68,1,1,1,1,0,1,1
77,0,0,0,0,0,1,1


So, these tables contain a timestamp_z. The (y, z) suffixes are applied to the left side of the join in `s3_combine_intra_subject_data.py`. We'll get `_z` when a subject has a treatment file but not a device status file. Manually looking at the files, I verified that each of these subjects matches that description. Therefore, we can select any of `[timestamp, timestamp_y, timestamp_z]` as the correct timestamp and have a timestamp for every table.

## Generalized Compression Design
The first few cells test a wrapper used for aggregation so we can get some stats out of the compression process. Then, I import a class I made to do this process and test it.

In [119]:
class Tester:
    
    def __init__(self):
        self.data = pd.DataFrame({'agg_col': [np.floor(i/5) for i in range(100)], 'vals': [i for i in range(100)]})
        self.carb_count = {'sum': 0, 'first': 0}  
    
    def test(self):
        return self.data.groupby('agg_col').agg({'vals': self.select_wrap})
    

    def select_wrap(self, x):
        x = self.select_carbs(x, self.carb_count)
        return x
#         @wraps(func)
#         def wrap(*args, **kwargs):
#             x, d = func(*args, **kwargs)
#             return x
#         return wrap
    
    @staticmethod
    def select_carbs(x, d):
#         print(f"x={x}, d= {d}")
        d['sum'] +=1
        return sum(x)

In [120]:
tester = Tester()
tester.test()
print(tester.carb_count)

{'sum': 20, 'first': 0}


Getting 

In [8]:
path = "C:\\Users\\spenc\\Documents\\Berkeley\\Capstone\\intrasubject_joinv0.2\\22961398_joined_csv"
subj = SubjectCompressor(path)

C:\Users\spenc\Documents\Berkeley\Capstone\BGPredict\bgpredict\bgpredict\intrasubject_compression.py:68: DtypeWarning: Columns (7,8,17,18,136,137,138,147,263,267,271,272,275,276,277,279,377,383,390,394,398,400,402,411,415,418,419,421,422,425,426,427) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(file)


In [11]:
conn = psycopg2.connect(
    host="bgpredict-db.cpfgoi91tnmw.us-west-2.rds.amazonaws.com",
    port=int(5432),
    user='postgres',
    password=
)

In [9]:
cursor = conn.cursor()
test= cursor.execute("""
SELECT *
FROM pg_catalog.pg_tables""")
conn.commit()
print(test)

None


In [13]:
cursor = conn.cursor()
test= cursor.execute("""
SELECT *
FROM pg_catalog.pg_tables""")

In [14]:
for table in cursor.fetchall():
    print(table)

('pg_catalog', 'pg_statistic', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_type', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_foreign_table', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_authid', 'rdsadmin', 'pg_global', True, False, False, False)
('pg_catalog', 'pg_statistic_ext_data', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_largeobject', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_user_mapping', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_subscription', 'rdsadmin', 'pg_global', True, False, False, False)
('pg_catalog', 'pg_attribute', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_proc', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_class', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_attrdef', 'rdsadmin', None, True, False, False, False)
('pg_catalog', 'pg_constraint', 'rdsadmin', None, True, False, False, Fa